# Covid-19 Analysis Worldwide
What we do
1. Import Data from the ECDC
2. Extend Data for Analysis
3. Do some Analysis
    1. Covid-19 Outbreak Comparison
    2. Top X Countries in Covid-19 Cases
    3. Top X Countries in Covid-19 Deaths
    4. Top X Countries in Covid-19 Growth Rates
    5. Covid-19 Growth Rates of Selected Countries
    6. Growth Rates over Time for Selected Countries
    7. Accumulated Cases over Time for Specific Countries
    8. Deaths over Cases for Specific Countries

In [ ]:
import requests
from requests_ntlm import HttpNtlmAuth

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib.pyplot as plt
import numpy as np

## Import Data
Data is imported from the ECDC (European Centre for Disease Prevention and Control - An Agency of the European Union)

In [ ]:
from datetime import datetime, timedelta
now =  datetime.now()
yesterday = now - timedelta(days=1)

In [ ]:
urlBase = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"
fileDate = now.strftime ("%Y-%m-%d")
url = urlBase + fileDate + ".xlsx"
r = requests.get (url, auth = HttpNtlmAuth(':',':'))
if r.status_code != 200:
    fileDate = yesterday.strftime("%Y-%m-%d") 
    url = urlBase + fileDate + ".xlsx" 
    r = requests.get (url, auth = HttpNtlmAuth(':',':'))
if r.status_code == 200:
    dataFileName = "tmpfile-" + fileDate + ".xlsx"
    dataFile = open (dataFileName, "wb")
    n = dataFile.write (r.content)
    dataFile.close()    

print ("Data loaded: " + url)    
print ("File Date: " + fileDate)


In [ ]:
df = pd.read_excel (dataFileName)
print ("Columns: " + ', '.join(df.columns))

In [ ]:
df.set_axis (["DateRep", "day", "month", "year", 
              "Cases", "Deaths", "Countries and territories", 
              "GeoId", "countryterritoryCode", "popData2018", "continentExp"], 
               axis = 1, inplace = True)
analysisDate = df["DateRep"].astype(str).max()
print ("Date Range: " + 
       df["DateRep"].astype(str).min() + 
       ", " + analysisDate
       )

# do some data fixing
i = 0
for id in df["GeoId"]:
    if not isinstance (id, str):
        df.at[i, "GeoId"] = df.loc[i, "countryterritoryCode"]
    i = i + 1
    
unique = df["GeoId"].unique()
index = list(map (lambda x: isinstance (x, str), unique))
unique = unique[index]
uniqueCount = unique.size

print ("Number of Countries: " + str(uniqueCount))
print ("Countries: " + ', '.join(unique))

## Prepare Data

In [ ]:
df.set_index("GeoId", inplace=True)

In [ ]:
df.sort_values(by=["GeoId", "DateRep"], inplace=True)

In [ ]:
df["Accumulated Cases"] = df.groupby(["GeoId"])["Cases"].cumsum()
df["Accumulated Deaths"] = df.groupby(["GeoId"])["Deaths"].cumsum()
df["Growth Factor Cases"] = 1 + df["Cases"] / (df["Accumulated Cases"] - df["Cases"])
df["Growth Factor Deaths"] = 1 + df["Deaths"] / (df["Accumulated Deaths"] - df["Deaths"])
df["Time to Double"] = np.log(2) / np.log(df["Growth Factor Cases"])

## Do some Analysis

### Covid-19 Outbreak Comparison with Germany as Reference

In [ ]:
cases = np.concatenate((np.array([50, 100, 500, 1000]),
                       np.arange (2500, 180000, 2500)))

#plotCountries = ["US", "IT", "FR"]
plotCountries = ["US", "IT", "ES", "UK", "IN"]

dict = {}
for i in cases:
    index = df["Accumulated Cases"] >= i
    results = df[index].groupby (["GeoId"])["DateRep"].min()
    dict[i] = results
delta = {}

m = len(cases)
n = len(plotCountries)+2
dfColumns = ["Cases", "DEDate"]
dfColumns = dfColumns + plotCountries
comparisonMatrix = pd.DataFrame (np.empty((m,n)),
                                columns = dfColumns)
i = 0
for case in cases:
    j = "DE"
    comparisonMatrix.at [i, "Cases"] =  case
    comparisonMatrix.at [i, "DEDate"] = dict[case]["DE"]
    for k in plotCountries:
        diff = dict[case]["DE"] - dict [case][k]
        x = diff.days
        comparisonMatrix.at [i, k] = x
    i = i + 1
    
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
for i in plotCountries:
    ax.plot(comparisonMatrix[i], comparisonMatrix["Cases"], label = i)
ax.set_title ("Covid-19 Outbreak Comparison with Germany in Accumulated Cases")
ax.set_axisbelow(True)
#ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
#ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

leg = ax.legend()


### Top X Countries in Covid-19 Cases

In [ ]:
topX = 20
topXCountriesInCases = (df.groupby(["Countries and territories"])["Cases"].sum().
 sort_values(ascending = False).head(topX))
countriesCases = topXCountriesInCases.keys()
fig1, ax1 = plt.subplots()
y_pos = np.arange(len(countriesCases))
ax1.barh(y_pos, topXCountriesInCases)
ax1.set_yticks(y_pos)
ax1.set_yticklabels(countriesCases)
ax1.set_title("Top " + str(topX) + " Countries in Covid-19 Cases on " + analysisDate)

plt.show()

### Top X Countries in Covid-19 Deaths

In [ ]:
topX = 10
topXCountriesInDeaths = (df.groupby(["Countries and territories"])["Deaths"].sum().
 sort_values(ascending = False).head(topX))
countriesDeaths = topXCountriesInDeaths.keys()
fig2, ax2 = plt.subplots()
y_pos = np.arange(len(countriesDeaths))
ax2.barh(y_pos, topXCountriesInDeaths)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(map(lambda a, b: a + " (" + str(b) + ")", 
                        countriesDeaths, topXCountriesInDeaths))
ax2.set_title("Top " + str(topX) + " Countries in Covid-19 Deaths on " + analysisDate)

plt.show()

### Top X Countries in Covid-19 Growth Rates

In [ ]:
threshhold = 500
topX = 20
topXCountriesGrowthFactor = (df.set_index("DateRep").
    loc[analysisDate].
    loc[lambda df: df["Accumulated Cases"] > threshhold].
    sort_values(by=["Growth Factor Cases"], 
                ascending = False)[["Countries and territories", 
                                    "Growth Factor Cases", "Accumulated Cases"]].
    head(topX))
countriesGrowthFactor = topXCountriesGrowthFactor["Countries and territories"]
fig3, ax3 = plt.subplots()
y_pos = np.arange(len(countriesGrowthFactor))
ylabels = map (lambda a, b: a + " (" + str(b) + ")", countriesGrowthFactor, topXCountriesGrowthFactor["Accumulated Cases"])
ax3.barh(y_pos, topXCountriesGrowthFactor["Growth Factor Cases"])
ax3.set_yticks(y_pos)
ax3.set_xlim(0.9, 1.5)
ax3.set_axisbelow(True)
#ax3.minorticks_on()
ax3.grid(which='major', linestyle='-', linewidth='0.5', color='black')
#ax3.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

ax3.set_yticklabels(ylabels)
ax3.set_title("Top " + str(topX) + " Countries in Covid-19 Growth Factor with more than " + 
              str(threshhold) + 
              " cases on " + 
              analysisDate + 
              " (Total Cases in Brackets)")

plt.show()

### Covid-19 Growth Rates of Selected Countries

In [ ]:
plotCountries = ["DE", "US", "IT", "FR", "UK", "ES", "SE","IN"]
countriesGrowthFactor = (df.loc[plotCountries].set_index("DateRep").
    loc[analysisDate].
    sort_values(by=["Growth Factor Cases"], ascending = False)[["Countries and territories", "Growth Factor Cases", "Accumulated Cases"]])
countries = countriesGrowthFactor["Countries and territories"]
fig4, ax4 = plt.subplots()
y_pos = np.arange(len(countriesGrowthFactor))
ylabels = map (lambda a, b: a + " (" + str(b) + ")", countries, countriesGrowthFactor["Accumulated Cases"])
ax4.barh(y_pos, countriesGrowthFactor["Growth Factor Cases"])
ax4.set_yticks(y_pos)
ax4.set_yticklabels(ylabels)
ax4.set_axisbelow(True)
ax4.minorticks_on()
ax4.grid(which='major', linestyle='-', linewidth='0.5', color='black')
ax4.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

ax4.set_xlim(0.9,1.2)
ax4.set_title("Selected Countries in Covid-19 Growth Factor on " + 
              analysisDate + 
              " (Total Cases in Brackets)")

plt.show()

### Growth Rates over Time for Selected Countries

In [ ]:
afterDate = df["DateRep"] >= "2020-03-01" 
plotCountries = ["DE", "US", "IT", "FR", "UK", "ES"]
#plotCountries = ["DE", "US"]
plotData = ["Time to Double"]
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
for j in plotData:
    for i in plotCountries:
        ax.plot(df[afterDate].loc[i, "DateRep"], df[afterDate].loc[i, j], label = i + ": " + j)
ax.set_title ("Covid-19 Analysis: " + analysisDate)
ax.set_ylim(0,60)
ax.set_axisbelow(True)
#ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
#ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

leg = ax.legend()

## Accumulated Cases over Time for Specific Countries

In [ ]:
afterDate = df["DateRep"] >= "2019-12-31" 
plotCountries = ["DE", "US", "IT", "FR", "UK", "ES", "SE", "IN"]
plotData = ["Accumulated Cases"]
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
for j in plotData:
    for i in plotCountries:
        ax.plot(df[afterDate].loc[i, "DateRep"], df[afterDate].loc[i, j], label = i + ": " + j)
ax.set_title ("Covid-19 Analysis: " + analysisDate)
ax.set_axisbelow(True)
#ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
#ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

leg = ax.legend()

## Deaths over Cases for Specific Countries

In [ ]:
plotCountries = ["DE", "US", "IT", "FR", "UK", "ES", "SE", "BE", "CN", "NL", "IN"]
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
for i in plotCountries:
    ax.plot(df.loc[i, "Accumulated Cases"], df.loc[i ,"Accumulated Deaths"], label = i)
ax.set_title ("Covid-19 Analysis: Deaths over Cases")
ax.set_axisbelow(True)
#ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black')
#ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
leg = ax.legend()